In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from itops.config.configs import CONFIGS

In [ ]:
df_themes = pd.read_csv("ITSM_Data_THEMES_ONLY.csv")

In [ ]:
from sentence_transformers import SentenceTransformer
MODEL_NAME = "all-MiniLM-L6-v2"

In [ ]:
def get_embedding_query_vector(query,model_name):
        """Get the vector of the query

        Args:
            query (string): user input

        Returns:
            _type_: vector of the query
        """
        model = SentenceTransformer(model_name)
        query_vector = model.encode(query)
        return query_vector

In [ ]:
len(df_themes)

In [ ]:
embedding_list = []
for i in range(len(df_themes)):
    content = df_themes.iloc[i]["themes"]
    embedding = get_embedding_query_vector(content,MODEL_NAME)
    embedding_list.append(embedding)
    print(f"Completed EMBEDDING  {i+1} ROW")

In [ ]:
type(embedding_list)

In [ ]:
df_themes["embeddings"] = embedding_list

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0)
embedding_array = np.array(df_themes["embeddings"].tolist())

kmeans.fit(embedding_array)

In [ ]:
labels = kmeans.labels_
df_themes["CLUSTERS"] = labels
df_themes.to_csv("CLUSTERS001.csv",index = False)

In [ ]:
num_clusters = 5
df = df_themes
user_input = "Please extract the theme of the provided context in Maximum 5 words"


In [ ]:
from itops.llm.azureopenaimanager.azure_open_ai_helper import AzureOpenAIManager
azure_open_ai_helper = AzureOpenAIManager(endpoint=CONFIGS.AZURE_OPENAI_ENDPOINT,
                                          api_key =CONFIGS.AZURE_OPENAI_API_KEY,
                                          deployment_id=CONFIGS.AZURE_OPENAI_DEPLOYMENT_ID,
                    api_version="2023-05-15")

In [ ]:
cluster_name_list = []
for i in range(num_clusters):
    df_cluster = df[df["CLUSTERS"] == i]
    full_content = ""
    for j in range(len(df_cluster)):
        content = df_cluster.iloc[j]["Text"]
        full_content = full_content + str(content) + " \n "
    
    cluster_name = azure_open_ai_helper.generate_reply_from_context(user_input, 
                                                            full_content, 
                                                            conversation=[])
    cluster_name_list.append(cluster_name[0])
    cluster_name =""
    print(f"Completed CLUSTERING PART 1 {i+1} ROW")


In [ ]:
cluster_name_list

In [ ]:
cluster_number_list= []
for i in range(len(cluster_name_list)):
    cluster_number_list.append(i)
df_clusters = pd.DataFrame()
df_clusters["CLUSTERS"] = cluster_number_list
df_clusters["CLUSTER_NAMES"] = cluster_name_list

In [ ]:
df_clusters

In [ ]:
df.columns

In [ ]:
df_all = df.merge(df_clusters)
print(f"Completed CLUSTERING FINAL")

In [ ]:
df_all